# Test cases

Cases below are working as designed.
Better avenue for test cases: rather than an arbitrary tree, use a sequence of intervals/rectangles/boxes added and subtracted.
Test these on gridded data to validate the simplifier and data filtering.

In [ ]:
from split_query.simplify import simplify_tree
from split_query.domain import simplify_domain
from split_query.expressions import And, Or, Not, Eq, Le, Lt, Ge, Gt, Float
from split_query.truth_table import expand_dnf

def pprint(result):
    print(result.expr)
    for cl in result.clauses:
        print(cl)

In [ ]:
def showoff(expression):
    pprint(expression)
    print()
    result = simplify_tree(simplify_domain(expand_dnf(simplify_domain(simplify_tree(expression)))))
    pprint(result)

In [ ]:
x, y = (Float(name) for name in 'xy')
showoff(And([
    And([Ge(x, 0), Le(x, 2), Ge(y, 0), Le(y, 2)]),
    Not(And([Ge(x, 0), Le(x, 1), Ge(y, 0), Le(y, 1)])),
]))

In [ ]:
showoff(And([Ge(x, 0), Le(x, 2), Ge(y, 0), Le(y, 2)]))

In [ ]:
showoff(And([
    And([Ge(x, 0), Le(x, 2), Ge(y, 0), Le(y, 2)]),
    Not(And([Ge(x, -1), Le(x, 1), Ge(y, -1), Le(y, 1)])),
]))